# My RNN Language Model implementation in Keras

In [2]:
import os

import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model
print(tf.__version__)

2.0.0


In [3]:
with open('../datasets/username_dataset.txt', 'r') as file:
    data = file.readlines()
    
data = [example.strip().lower() for example in data]

with open('../datasets/username_dataset.txt', 'r') as file:
    data_str = file.read().lower()

vocab = list(set(data_str))
vocab.sort()

vocab.insert(0, '>') # End token: <end>
vocab.insert(0, '<') # Start token: <start>
vocab.insert(0, '*') # Padding

char_to_index = {ch: i for i, ch in enumerate(vocab)}
index_to_char = {i: ch for i, ch in enumerate(vocab)}

print(char_to_index)

longest_word = max(data, key=len)

print(f"Number of characters: {len(data_str)}")
print(f"Number of unique characters: {len(vocab)}")
print(f"Longest word: {longest_word}")
print(f"Length of longest word: {len(longest_word)}") # Needed information to know what length to pad the usernames to.

{'*': 0, '<': 1, '>': 2, '\n': 3, '0': 4, '1': 5, '2': 6, '3': 7, '4': 8, '5': 9, '6': 10, '7': 11, '8': 12, '9': 13, 'a': 14, 'b': 15, 'c': 16, 'd': 17, 'e': 18, 'f': 19, 'g': 20, 'h': 21, 'i': 22, 'j': 23, 'k': 24, 'l': 25, 'm': 26, 'n': 27, 'o': 28, 'p': 29, 'q': 30, 'r': 31, 's': 32, 't': 33, 'u': 34, 'v': 35, 'w': 36, 'x': 37, 'y': 38, 'z': 39}
Number of characters: 401244
Number of unique characters: 40
Longest word: rabbitsreviews
Length of longest word: 14


In [4]:
def print_mapping(mapping):
    print('{')
    for char, _ in zip(mapping, range(20)):
        print('  {:4s}: {:3d},'.format(repr(char), mapping[char]))
    print('  ...\n}')
    
print_mapping(char_to_index)

{
  '*' :   0,
  '<' :   1,
  '>' :   2,
  '\n':   3,
  '0' :   4,
  '1' :   5,
  '2' :   6,
  '3' :   7,
  '4' :   8,
  '5' :   9,
  '6' :  10,
  '7' :  11,
  '8' :  12,
  '9' :  13,
  'a' :  14,
  'b' :  15,
  'c' :  16,
  'd' :  17,
  'e' :  18,
  'f' :  19,
  ...
}


In [5]:
def prepare_dataset(data, username_length):
    """Adds <start> and <end> tokens and pads the usernames."""
    data_X = []
    data_Y = []
    for username in data:
        pad = "*" * (username_length - len(username))
        X = np.array([char_to_index[char] for char in f"<{username}{pad}"]).reshape((username_length + 1, 1))
        Y = np.array([char_to_index[char] for char in f"{username}>{pad}"]).reshape((username_length + 1, 1))
        data_X.append(X)
        data_Y.append(Y)
    

    return (np.array(data_X), np.array(data_Y))

data = prepare_dataset(data, len(longest_word))

print(data[0].shape)
print(data[1].shape)
dataset = tf.data.Dataset.from_tensor_slices(data)

(54037, 15, 1)
(54037, 15, 1)


In [6]:
for input_example, target_example in dataset.take(1):
    print('Input data: ', repr(''.join([index_to_char[i] for i in input_example[:, 0].numpy()])))
    print('Target data:', repr(''.join([index_to_char[i] for i in target_example[:, 0].numpy()])))
    
for input_example, target_example in dataset.take(1):
    print('Input data: ', input_example.numpy())
    print('Target data:', target_example.numpy())

Input data:  '<info**********'
Target data: 'info>**********'
Input data:  [[ 1]
 [22]
 [27]
 [19]
 [28]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]]
Target data: [[22]
 [27]
 [19]
 [28]
 [ 2]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]]


In [7]:
BATCH_SIZE = 128
BUFFER_SIZE = 10000
EPOCHS = 100

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
print(dataset)

<BatchDataset shapes: ((128, 15, 1), (128, 15, 1)), types: (tf.int32, tf.int32)>


In [8]:
vocab_size = len(vocab)
rnn_units = 512

In [9]:
def build_model(vocab_size, rnn_units, batch_size, stateful=False):
    model = tf.keras.Sequential([
        tf.keras.layers.Masking(mask_value=0, batch_input_shape=[batch_size, None, 1]),
        tf.keras.layers.LSTM(rnn_units,
                             return_sequences=True,
                             stateful=stateful,
                             recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size, activation=tf.nn.softmax)])
    
    return model

In [10]:
model = build_model(
    vocab_size = len(vocab),
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (128, None, 1)            0         
_________________________________________________________________
lstm (LSTM)                  (128, None, 512)          1052672   
_________________________________________________________________
dense (Dense)                (128, None, 40)           20520     
Total params: 1,073,192
Trainable params: 1,073,192
Non-trainable params: 0
_________________________________________________________________


In [12]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

In [13]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss',
                                               min_delta=0,
                                               patience=5,
                                               verbose=0,
                                               mode='auto',
                                               baseline=None,
                                               restore_best_weights=True)

In [13]:
model.fit(dataset, epochs=EPOCHS, callbacks=[early_stopping])

Epoch 1/100
422/422 [==============================] - 72s 172ms/step - loss: 1.3925
Epoch 2/100
422/422 [==============================] - 63s 150ms/step - loss: 1.3436
Epoch 3/100
422/422 [==============================] - 61s 144ms/step - loss: 1.3328
Epoch 4/100
422/422 [==============================] - 59s 139ms/step - loss: 1.3231
Epoch 5/100
422/422 [==============================] - 59s 140ms/step - loss: 1.3146
Epoch 6/100
422/422 [==============================] - 64s 152ms/step - loss: 1.3062
Epoch 7/100
422/422 [==============================] - 59s 140ms/step - loss: 1.2982
Epoch 8/100
422/422 [==============================] - 61s 144ms/step - loss: 1.29001s - l
Epoch 9/100
422/422 [==============================] - 61s 145ms/step - loss: 1.2810
Epoch 10/100
422/422 [==============================] - 60s 143ms/step - loss: 1.2726
Epoch 11/100
422/422 [==============================] - 60s 143ms/step - loss: 1.2643
Epoch 12/100
422/422 [==============================] - 6

In [14]:
# model.save_weights('../weights/lstm_model_weights.h5')

In [14]:
model = build_model(vocab_size, rnn_units, batch_size=1, stateful=True)

model.load_weights('../weights/lstm_model_weights.h5')

# model.build(tf.TensorShape([1, None, 1]))

In [15]:
def generate_usernames(model):
    num_generate = 7
    generated_usernames = []

    for i in range(num_generate):
        model.reset_states()
        input_eval = np.array([char_to_index['<']]).reshape((1, 1, 1))
        generated_username = []
        done = False
        
        while not done:
            predictions = model.predict(input_eval)
            predictions = tf.squeeze(predictions)

            predicted_id = np.random.choice(range(40), p=predictions.numpy())
            # predicted_id = np.argmax(predictions)

            input_eval = np.array([predicted_id]).reshape((1, 1, 1))
            
            done = index_to_char[predicted_id] in ['>', '*']
            
            if not done:
                generated_username.append(index_to_char[predicted_id])
        generated_usernames.append("".join(generated_username))

    return generated_usernames

In [16]:
for _ in range(20):
    generated_usernames = generate_usernames(model)
    print(generated_usernames)

['ventel', 'broeer', 'mushtoop', 'dosk', 'somoco', 'toflanp', 'blacer']
['olenic', 'arvisu', 'heklomip', 'tter', 'spnyrginar', 'miqtx', 'kbrbmner']
['stretch1', 'bosexavnr', 'moiiem', 'ruggo1', 'sacison', 'cicuinne', 'sieefy']
['garthw', 'chaoges', 'gererhf', 'sptics', 'kahal', 'rlxdavg', 'pane1100']
['fredlan', 'stdbhthf', 'tuail', 'dinar', 'repley', 'dncrroh', 'seyglwg']
['siggen', 'dave69', 'sauan13', 'bearpy', 'trjcian', 'erocerah', 'duckmunt']
['argc', 'charliex', 'caryash', 'poinmo', 'tqucky', 'jakesne', 'psinter']
['topnap', 'slartupm', 'alland', 'cabl', 'heory123', 'fbonwas', 'jndsth']
['anad', 'henry69', 'fodnre', 'mboidy200', 'greyfox', 'pawlgwey', 'maddawg']
['retepe', 'petec', 'demlan', 'mhckey', 'gogfa', 'serti', 'eirayusr']
['shgnyb', 'saibot', 'swpod', 'mahike', 'nylonjan', 'tiezerk', '12vibh']
['oeily', 'mlldy', 'laconi', 'glole', 'jbhrl', 'fvln1133', 'ampter']
['bigmac', 'cearar', 'hamster1', 'buddy12', 'brady1', 'optifr1', 'dufavi1']
['bigcogc', 'rarta2', 'katmind', '